In [ ]:
# Cell 0 — GPU check, installs, reproducibility
# Make sure GPU is enabled in Kaggle: Settings -> Accelerator -> GPU

# Basic checks
!nvidia-smi

# Install kagglehub if not present (used to download the Kaggle dataset)
!pip install -q kagglehub

# TensorFlow (Kaggle typically has TF preinstalled). If you want a specific version, uncomment:
# !pip install -q "tensorflow==2.14.0"

# Reproducibility
import random, os, numpy as np, tensorflow as tf
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("TensorFlow version:", tf.__version__)

In [ ]:
# Cell 2 — Create project folders (Kaggle adapted)
import os

# Kaggle working directory
PROJECT_DIR = "/kaggle/working"

# Check for input dataset (standard Kaggle location)
KAGGLE_INPUT_DIR = "/kaggle/input/brain-tumor-mri-dataset"

# We will determine RAW_DIR dynamically. 
# If the dataset is added as input, use it. Otherwise, we'll download it to a temp location or working dir.
if os.path.exists(KAGGLE_INPUT_DIR):
    RAW_DIR = KAGGLE_INPUT_DIR
    print(f"Using Kaggle input dataset at: {RAW_DIR}")
else:
    # Fallback: will be handled in next cell via kagglehub
    RAW_DIR = os.path.join(PROJECT_DIR, "data", "raw")
    print(f"Dataset not found in input. Will attempt download in next cell.")

PROCESSED_DIR = os.path.join(PROJECT_DIR, "data", "processed")
MODEL_DIR = os.path.join(PROJECT_DIR, "models")
REPORT_DIR = os.path.join(PROJECT_DIR, "reports")
ARTIFACTS_DIR = os.path.join(PROJECT_DIR, "artifacts")

for d in [PROCESSED_DIR, MODEL_DIR, REPORT_DIR, ARTIFACTS_DIR]:
    os.makedirs(d, exist_ok=True)

# If RAW_DIR is inside working (i.e. we need to download it), create it
if RAW_DIR.startswith(PROJECT_DIR):
     os.makedirs(RAW_DIR, exist_ok=True)

print("Project directories set up.")

In [ ]:
# Cell 3 — Dataset setup
import shutil, os
import kagglehub

# Check if we need to download
# We check if RAW_DIR contains the expected 'Training' folder or if it's the input dir
if not os.path.exists(os.path.join(RAW_DIR, "Training")):
    print("Dataset not found in RAW_DIR. Downloading using kagglehub...")
    try:
        path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")
        print("Downloaded to:", path)
        RAW_DIR = path # Update RAW_DIR to point to the downloaded cache
        print(f"Updated RAW_DIR to: {RAW_DIR}")
    except Exception as e:
        print(f"Error downloading dataset: {e}")
else:
    print(f"Dataset already present at {RAW_DIR}")

# No need to zip and copy to Drive on Kaggle

In [ ]:
# Cell 4 — show folder tree & counts
from pathlib import Path

def walk(path):
    for root, dirs, files in os.walk(path):
        print(f"DIR: {root} | FILES: {len(files)}")
walk(RAW_DIR)

# List training/testing subfolders if present
from pprint import pprint
train_dir = os.path.join(RAW_DIR, "Training")
test_dir = os.path.join(RAW_DIR, "Testing")

if os.path.exists(train_dir):
    classes = sorted([p.name for p in Path(train_dir).iterdir() if p.is_dir()])
    print("Detected classes (Training):", classes)
    counts = {c: len(list(Path(train_dir).joinpath(c).glob("*"))) for c in classes}
    print("Training counts:", counts)
else:
    print("Training directory not found at:", train_dir)


In [ ]:
# Cell 5 — Create tf.data datasets (train and val/test)
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory

IMG_SIZE = (224, 224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE
SEED = 42

# Paths (assumes dataset has 'Training' and 'Testing' directories)
TRAIN_DIR = train_dir
VAL_DIR = test_dir

# If dataset is not split, you can use validation_split in image_dataset_from_directory
train_ds = image_dataset_from_directory(
    TRAIN_DIR,
    labels='inferred',
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=SEED
)

val_ds = image_dataset_from_directory(
    VAL_DIR,
    labels='inferred',
    label_mode='int',
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    shuffle=False
)

class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names, "Num classes:", num_classes)

# Data augmentation + preprocessing
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.08),
    layers.RandomZoom(0.08),
    layers.RandomTranslation(0.04, 0.04)
], name="data_augmentation")

preprocess_input = tf.keras.applications.efficientnet.preprocess_input

def prepare(ds, training=False):
    ds = ds.map(lambda x, y: (tf.image.resize(x, IMG_SIZE), y), num_parallel_calls=AUTOTUNE)
    if training:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=AUTOTUNE)
    ds = ds.map(lambda x, y: (preprocess_input(x), y), num_parallel_calls=AUTOTUNE)
    return ds.cache().prefetch(AUTOTUNE)

train_ds = prepare(train_ds, training=True)
val_ds = prepare(val_ds, training=False)


In [ ]:
# Cell 6 — Compute class weights using training folder counts
from collections import Counter
import numpy as np
train_counts = {c: len(list(Path(TRAIN_DIR).joinpath(c).glob("*"))) for c in class_names}
print("Train counts:", train_counts)
total = sum(train_counts.values())
class_weight = {i: total/(num_classes * train_counts[class_names[i]]) for i in range(num_classes)}
print("Class weights:", class_weight)


In [ ]:
# Cell 7 — Build hybrid model with PRESERVED SPATIAL GRADIENTS for Grad-CAM
import tensorflow as tf
from tensorflow.keras import layers, models

def transformer_encoder(inputs, head_size=64, num_heads=4, ff_dim=128, dropout=0.1):
    x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(num_heads=num_heads, key_dim=head_size)(x, x)
    x = layers.Dropout(dropout)(x)
    x = layers.Add()([x, inputs])
    y = layers.LayerNormalization(epsilon=1e-6)(x)
    y = layers.Dense(ff_dim, activation="relu")(y)
    y = layers.Dropout(dropout)(y)
    y = layers.Dense(inputs.shape[-1])(y)
    out = layers.Add()([x, y])
    return out

def build_hybrid_model_gradcam(input_shape=(224,224,3), num_classes=4, dropout_rate=0.3):
    """
    Modified architecture that preserves spatial gradient flow for Grad-CAM.
    
    Key changes from original:
    - Added 1x1 conv layers BEFORE GlobalAveragePooling
    - This creates a gradient path: output -> transformer -> GAP -> conv -> spatial features
    - The 'gradcam_target_conv' layer is designed as the Grad-CAM visualization target
    """
    inputs = layers.Input(shape=input_shape)
    base = tf.keras.applications.EfficientNetB0(include_top=False, input_tensor=inputs, weights='imagenet')
    base.trainable = False  # freeze initially

    # x shape: [B, 7, 7, 1280] from EfficientNetB0
    x = base.output
    
    # === GRAD-CAM FRIENDLY ARCHITECTURE ===
    # Apply 1x1 convolutions BEFORE pooling to maintain gradient path to spatial features
    
    # Primary Grad-CAM target layer - gradients flow here for visualization
    x = layers.Conv2D(512, (1, 1), activation='relu', name='gradcam_target_conv')(x)  # [B, 7, 7, 512]
    x = layers.BatchNormalization(name='gradcam_target_bn')(x)
    
    # Additional feature processing (still spatial)
    x = layers.Conv2D(256, (1, 1), activation='relu', name='feature_conv')(x)  # [B, 7, 7, 256]
    x = layers.BatchNormalization()(x)
    
    # GlobalAveragePooling - spatial info aggregated here
    # Grad-CAM computes gradients w.r.t. conv layers BEFORE this
    x = layers.GlobalAveragePooling2D(name='final_gap')(x)  # [B, 256]
    
    # Transformer on pooled features (single token)
    token_dim = 256
    x_tokens = layers.Reshape((1, x.shape[-1]))(x)  # [B, 1, 256]
    
    # Small transformer stack
    for _ in range(2):
        x_tokens = transformer_encoder(x_tokens, head_size=64, num_heads=4, ff_dim=512, dropout=0.1)
    
    # Classification head
    x_flat = layers.LayerNormalization()(x_tokens)
    x_flat = layers.Flatten()(x_flat)
    x_flat = layers.Dropout(dropout_rate)(x_flat)
    x_flat = layers.Dense(128, activation='relu')(x_flat)
    x_flat = layers.Dropout(dropout_rate)(x_flat)
    outputs = layers.Dense(num_classes, activation='softmax', name='predictions')(x_flat)

    model = models.Model(inputs, outputs, name="EffNetB0_TransformerHead_GradCAM")
    return model

# Build the new Grad-CAM friendly model
model = build_hybrid_model_gradcam(input_shape=IMG_SIZE + (3,), num_classes=num_classes)
model.summary()

print("\n" + "="*70)
print("🔧 MODEL ARCHITECTURE MODIFIED FOR PROPER GRAD-CAM")
print("="*70)
print("""
ARCHITECTURE CHANGES:
┌─────────────────────────────────────────────────────────────────────┐
│  EfficientNetB0 (frozen)                                            │
│       ↓ [B, 7, 7, 1280]                                            │
│  Conv2D 1x1 (512) → 'gradcam_target_conv'  ← GRAD-CAM TARGET       │
│       ↓ [B, 7, 7, 512]                                             │
│  Conv2D 1x1 (256) → 'feature_conv'                                  │
│       ↓ [B, 7, 7, 256]                                             │
│  GlobalAveragePooling2D                                             │
│       ↓ [B, 256]                                                   │
│  Transformer (2 blocks)                                             │
│       ↓                                                            │
│  Dense → Softmax                                                    │
└─────────────────────────────────────────────────────────────────────┘

WHY THIS WORKS FOR GRAD-CAM:
• Gradients flow: Output → Transformer → GAP → Conv layers → Spatial features
• The conv layers BEFORE GAP preserve the spatial information needed for Grad-CAM
• 'gradcam_target_conv' has shape [7, 7, 512] - perfect for spatial visualization

⚠️  IMPORTANT: This is a NEW model architecture - you need to RETRAIN!
    Run cells 8 (compile) and 9 (train) to train this new model.
""")
print("="*70)

In [ ]:
# # Cell 8 — compile & callbacks
# from tensorflow.keras import optimizers, callbacks

# EPOCHS = 20
# LR = 3e-4

# model.compile(
#     optimizer=optimizers.Adam(learning_rate=LR),
#     loss='sparse_categorical_crossentropy',
#     metrics=['accuracy']
# )

# os.makedirs(MODEL_DIR, exist_ok=True)
# ckpt_path = os.path.join(MODEL_DIR, "best_model.h5")

# cb_list = [
#     callbacks.ModelCheckpoint(ckpt_path, monitor='val_accuracy', save_best_only=True, verbose=1),
#     callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1),
#     callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
# ]

# Cell 8 — compile & callbacks
from tensorflow.keras import optimizers, callbacks

EPOCHS = 20
LR = 3e-4

model.compile(
    optimizer=optimizers.Adam(learning_rate=LR),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

os.makedirs(MODEL_DIR, exist_ok=True)

# ✔ Save model in new Keras format to avoid legacy HDF5 warning
ckpt_path = os.path.join(MODEL_DIR, "best_model.keras")

cb_list = [
    callbacks.ModelCheckpoint(
        filepath=ckpt_path,
        monitor='val_accuracy',
        save_best_only=True,
        save_weights_only=False,   # Use full model (recommended)
        mode='max',
        verbose=1
    ),
    
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=6,
        restore_best_weights=True,
        mode='min',
        verbose=1
    ),
    
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-6,
        mode='min',
        verbose=1
    )
]


In [ ]:
# Cell 9 — train
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight,
    callbacks=cb_list
)


In [ ]:
 # Cell 10 — Evaluate on validation set and compute classification report
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools

# collect predictions
y_true = []
y_pred = []
y_prob = []

for batch_x, batch_y in val_ds:
    preds = model.predict(batch_x)
    y_true.extend(batch_y.numpy().tolist())
    y_pred.extend(np.argmax(preds, axis=1).tolist())
    y_prob.extend(np.max(preds, axis=1).tolist())

print(classification_report(y_true, y_pred, target_names=class_names))
cm = confusion_matrix(y_true, y_pred)
print("Confusion matrix:\n", cm)

def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

plot_confusion_matrix(cm, class_names)


In [ ]:
# Cell 11 — save predictions with filenames to CSV for RAG ingestion
import csv
from pathlib import Path

# Build mapping of filenames to labels for validation set
# image_dataset_from_directory preserves order within batches but easier to re-walk val folder
rows = []
val_root = Path(VAL_DIR)
# Walk classes and files
for class_idx, cls in enumerate(class_names):
    p = val_root.joinpath(cls)
    for img_path in p.glob("*"):
        rows.append({"filepath": str(img_path), "true_label": cls})

# Note: val order here may not match the dataset order used above — so recompute preds by reading images file-by-file
def preprocess_image(path):
    img = tf.io.read_file(str(path))
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.resize(img, IMG_SIZE)
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)
    return img

out_rows = []
for r in rows:
    img = preprocess_image(r["filepath"])
    img_batch = tf.expand_dims(img, axis=0)
    preds = model.predict(img_batch)
    pred_idx = int(np.argmax(preds, axis=1)[0])
    pred_label = class_names[pred_idx]
    confidence = float(np.max(preds))
    out_rows.append([r["filepath"], r["true_label"], pred_label, confidence])

csv_path = os.path.join(ARTIFACTS_DIR, "val_predictions.csv")
with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["filepath", "true_label", "pred_label", "confidence"])
    writer.writerows(out_rows)

print("Saved predictions CSV to:", csv_path)


In [ ]:
# Cell 12 — Grad-CAM Setup for Modified Model
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess

CLASS_NAMES = class_names
print("CLASS_NAMES:", CLASS_NAMES)

# Preprocessing function
def load_and_preprocess_for_model(img_path):
    img = keras_image.load_img(img_path, target_size=IMG_SIZE)
    x = keras_image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = effnet_preprocess(x)
    return x

# === USE THE NEW GRAD-CAM TARGET LAYER ===
# The modified model has 'gradcam_target_conv' specifically designed for Grad-CAM
# This layer is BEFORE GlobalAveragePooling, so gradients flow properly

print("\nSearching for Grad-CAM target layer...")
try:
    # Use our new custom layer designed for Grad-CAM
    gradcam_layer = model.get_layer('gradcam_target_conv')
    print(f"✓ Found gradcam_target_conv: {gradcam_layer.name}")
except:
    # Fallback to EfficientNet's top_conv if using old model
    gradcam_layer = model.get_layer('top_conv')
    print(f"Using fallback top_conv: {gradcam_layer.name}")

print(f"Output shape: {gradcam_layer.output.shape}")

# Build Grad-CAM model: outputs conv features AND predictions
grad_model = tf.keras.Model(
    inputs=model.input,
    outputs=[gradcam_layer.output, model.output]
)

# Verify the model works
print("\nVerifying grad_model...")
dummy = np.zeros((1, 224, 224, 3), dtype=np.float32)
conv_out, pred_out = grad_model(dummy, training=False)
print(f"Conv output shape: {conv_out.shape}")  # Should be [1, 7, 7, 512]
print(f"Pred output shape: {pred_out.shape}")  # Should be [1, 4]
print("✓ Grad-CAM model ready!")

In [ ]:
# Cell 13 — Proper Gradient-Based Grad-CAM (works with modified architecture)
# Now that we have conv layers BEFORE GAP, gradients flow properly!

def make_gradcam_heatmap(img_batch, target_class_idx, debug=False):
    """
    Generate proper Grad-CAM heatmap using gradients.
    
    Works because our modified architecture has conv layers BEFORE GlobalAveragePooling,
    allowing gradients to flow from the output back to spatial feature maps.
    """
    img_batch = tf.cast(img_batch, tf.float32)
    
    with tf.GradientTape() as tape:
        # Forward pass - get conv outputs and predictions
        conv_outputs, predictions = grad_model(img_batch, training=False)
        
        # Watch conv_outputs for gradient computation
        tape.watch(conv_outputs)
        
        # Get the score for the target class
        class_score = predictions[:, target_class_idx]
        
        if debug:
            print(f"  Target class: {target_class_idx} ({CLASS_NAMES[target_class_idx]})")
            print(f"  Class score: {class_score.numpy()[0]:.4f}")
    
    # Compute gradients of class score w.r.t. conv outputs
    grads = tape.gradient(class_score, conv_outputs)
    
    if grads is None:
        print("ERROR: Gradients are None!")
        print("This means gradient flow is still broken.")
        print("Make sure you're using the modified model architecture.")
        # Return a uniform heatmap as fallback
        return np.ones((7, 7), dtype=np.float32) * 0.5
    
    if debug:
        print(f"  ✓ Gradients computed! Shape: {grads.shape}")
        print(f"  Gradient range: [{grads.numpy().min():.6f}, {grads.numpy().max():.6f}]")
    
    # Global Average Pool the gradients to get channel importance weights
    # Shape: [1, 7, 7, channels] -> [1, channels]
    pooled_grads = tf.reduce_mean(grads, axis=(1, 2))
    
    # Weight the conv outputs by the gradient importance
    conv_outputs = conv_outputs[0]  # Remove batch dim: [7, 7, channels]
    pooled_grads = pooled_grads[0]  # Remove batch dim: [channels]
    
    # Weighted combination: each channel weighted by its gradient importance
    heatmap = tf.reduce_sum(conv_outputs * pooled_grads, axis=-1)  # [7, 7]
    heatmap = heatmap.numpy()
    
    # Apply ReLU - we only care about features that positively affect the class
    heatmap = np.maximum(heatmap, 0)
    
    # Normalize
    if heatmap.max() > 0:
        heatmap = heatmap / heatmap.max()
    
    if debug:
        print(f"  Heatmap range: [{heatmap.min():.3f}, {heatmap.max():.3f}]")
        if heatmap.max() > 0:
            max_pos = np.unravel_index(np.argmax(heatmap), heatmap.shape)
            print(f"  Max activation at: {max_pos}")
    
    return heatmap


def resize_heatmap_to_image(heatmap, target_size):
    """Resize 7x7 heatmap to full image size with smoothing."""
    # Resize using bilinear interpolation
    heatmap_resized = tf.image.resize(
        heatmap[..., np.newaxis], 
        target_size, 
        method='bilinear'
    )
    heatmap_resized = tf.squeeze(heatmap_resized, axis=-1).numpy()
    heatmap_resized = np.clip(heatmap_resized, 0, 1)
    
    # Apply slight smoothing for better visualization
    heatmap_tf = tf.constant(heatmap_resized[None, ..., None], dtype=tf.float32)
    heatmap_smooth = tf.nn.avg_pool2d(heatmap_tf, ksize=5, strides=1, padding='SAME')
    heatmap_smooth = tf.squeeze(heatmap_smooth).numpy()
    
    if heatmap_smooth.max() > 0:
        heatmap_smooth = heatmap_smooth / heatmap_smooth.max()
    
    return heatmap_smooth


# === TEST GRADIENT FLOW ===
print("Testing gradient-based Grad-CAM...")
print("="*50)

test_input = np.random.randn(1, 224, 224, 3).astype(np.float32)
test_heatmap = make_gradcam_heatmap(test_input, 0, debug=True)

print("="*50)
if test_heatmap.max() > 0.5 or test_heatmap.min() != test_heatmap.max():
    print("✓ SUCCESS! Gradients are flowing properly!")
    print(f"  Heatmap shape: {test_heatmap.shape}")
    print("  Grad-CAM is working correctly with the modified architecture.")
else:
    print("⚠ Heatmap may be uniform - check if gradients are valid.")

In [ ]:
# Main visualization function - Shows ONE heatmap for predicted class
def visualize_gradcam_single(img_path, true_label=None, alpha=0.6, save_path=None):
    """
    Generate and display a single Grad-CAM heatmap for the predicted class.
    Shows what regions the model focuses on to make its prediction.
    """
    # Load and preprocess image
    img_batch = load_and_preprocess_for_model(img_path)
    
    # Get model prediction
    predictions = model.predict(img_batch, verbose=0)[0]
    pred_idx = int(np.argmax(predictions))
    pred_label = CLASS_NAMES[pred_idx]
    pred_confidence = float(predictions[pred_idx])
    
    # Generate Grad-CAM heatmap for the PREDICTED class
    heatmap_lowres = make_gradcam_heatmap(img_batch, pred_idx)
    print(f"  Low-res heatmap shape: {heatmap_lowres.shape}, range: [{heatmap_lowres.min():.2f}, {heatmap_lowres.max():.2f}]")
    
    heatmap = resize_heatmap_to_image(heatmap_lowres, IMG_SIZE)
    
    # Load original image for display (NOT preprocessed)
    original_img = keras_image.load_img(img_path, target_size=IMG_SIZE)
    original_img = keras_image.img_to_array(original_img) / 255.0
    
    # Determine if it's a tumor or not
    is_tumor = pred_label.lower() != 'notumor'
    
    # Create figure
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    
    # 1. Original image
    axes[0].imshow(original_img)
    axes[0].set_title('Original MRI Scan', fontsize=12, fontweight='bold')
    axes[0].axis('off')
    
    # 2. Heatmap visualization
    if is_tumor:
        # For tumor: show heatmap highlighting tumor region with 'hot' colormap
        im = axes[1].imshow(heatmap, cmap='jet', vmin=0, vmax=1)
        axes[1].set_title(f'Tumor Location ({pred_label})', fontsize=12, fontweight='bold', color='darkred')
        
        # Overlay - show tumor region on MRI
        axes[2].imshow(original_img)
        # Use threshold to show only high-activation areas
        heatmap_thresh = np.where(heatmap > 0.3, heatmap, 0)
        axes[2].imshow(heatmap_thresh, cmap='jet', alpha=alpha, vmin=0, vmax=1)
        axes[2].set_title('Tumor Region Highlighted', fontsize=12, fontweight='bold', color='darkred')
    else:
        # For no tumor: show what model examined
        im = axes[1].imshow(heatmap, cmap='Greens', vmin=0, vmax=1)
        axes[1].set_title('Model Focus Area', fontsize=12, fontweight='bold', color='darkgreen')
        
        axes[2].imshow(original_img)
        axes[2].imshow(heatmap, cmap='Greens', alpha=alpha * 0.6, vmin=0, vmax=1)
        axes[2].set_title('No Tumor - Healthy Scan', fontsize=12, fontweight='bold', color='darkgreen')
    
    axes[1].axis('off')
    axes[2].axis('off')
    plt.colorbar(im, ax=axes[1], fraction=0.046, pad=0.04)
    
    # Build main title
    if true_label is not None:
        correct = "✓ Correct" if true_label == pred_label else "✗ Incorrect"
        main_title = f"{correct} | True: {true_label} | Predicted: {pred_label} ({pred_confidence:.1%})"
        title_color = 'green' if true_label == pred_label else 'red'
    else:
        main_title = f"Predicted: {pred_label} ({pred_confidence:.1%})"
        title_color = 'darkred' if is_tumor else 'darkgreen'
    
    fig.suptitle(main_title, fontsize=14, fontweight='bold', color=title_color)
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        print(f"Saved to: {save_path}")
    
    plt.show()
    
    # Print class probabilities
    print(f"\nClass Probabilities:")
    for idx, class_name in enumerate(CLASS_NAMES):
        prob = predictions[idx]
        bar = '█' * int(prob * 30)
        marker = " ← Predicted" if idx == pred_idx else ""
        print(f"  {class_name:12s}: {prob:.4f} {bar}{marker}")
    
    # Print interpretation
    if is_tumor:
        print(f"\n⚠️  TUMOR DETECTED: {pred_label.upper()}")
        print(f"   The highlighted regions show where the model identified tumor characteristics.")
    else:
        print(f"\n✓  NO TUMOR DETECTED")
        print(f"   The model examined the brain tissue and found no signs of tumor.")
    
    return pred_label, pred_confidence, heatmap

In [ ]:
# Batch visualization from validation set
def visualize_batch_gradcam(num_samples=6, include_all_classes=True):
    """
    Visualize Grad-CAM for multiple samples from each class.
    
    Args:
        num_samples: Total number of samples to visualize
        include_all_classes: If True, sample from each class for diversity
    """
    import os
    import random
    
    # Collect images from training/validation directory
    data_dir = VAL_DIR if os.path.exists(VAL_DIR) else TRAIN_DIR
    
    if include_all_classes:
        # Sample from each class for diversity
        samples_per_class = max(1, num_samples // len(CLASS_NAMES))
        sampled = []
        
        for cls_name in CLASS_NAMES:
            cls_path = os.path.join(data_dir, cls_name)
            if not os.path.isdir(cls_path):
                continue
            
            images = [f for f in os.listdir(cls_path) 
                     if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            
            if images:
                selected = random.sample(images, min(samples_per_class, len(images)))
                for img_name in selected:
                    sampled.append((os.path.join(cls_path, img_name), cls_name))
        
        # Shuffle and limit to requested number
        random.shuffle(sampled)
        sampled = sampled[:num_samples]
    else:
        # Random sampling across all classes
        sampled = []
        for cls_name in CLASS_NAMES:
            cls_path = os.path.join(data_dir, cls_name)
            if os.path.isdir(cls_path):
                for img_name in os.listdir(cls_path):
                    if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                        sampled.append((os.path.join(cls_path, img_name), cls_name))
        
        sampled = random.sample(sampled, min(num_samples, len(sampled)))
    
    print(f"Generating Grad-CAM visualizations for {len(sampled)} images...")
    print("=" * 70)
    
    results = []
    for idx, (img_path, true_label) in enumerate(sampled, 1):
        print(f"\n[{idx}/{len(sampled)}] {os.path.basename(img_path)}")
        pred_label, confidence, heatmap = visualize_gradcam_single(img_path, true_label)
        results.append({
            'path': img_path,
            'true_label': true_label,
            'pred_label': pred_label,
            'confidence': confidence,
            'correct': true_label == pred_label
        })
        print("-" * 70)
    
    # Summary
    correct = sum(1 for r in results if r['correct'])
    print(f"\n{'='*70}")
    print(f"Summary: {correct}/{len(results)} correct predictions ({100*correct/len(results):.1f}% accuracy)")
    
    return results


# Run visualization
print("Generating Grad-CAM visualizations for brain tumor classification...")
print("Each visualization shows ONE heatmap for the predicted class.\n")

results = visualize_batch_gradcam(num_samples=6, include_all_classes=True)

In [ ]:
# Test on a single specific image
# Replace the path with any image you want to analyze

# Example: using a path from the validation set
import os
import random

# Find an example image
example_path = None
if os.path.exists(VAL_DIR):
    for cls_name in class_names:
        cls_path = os.path.join(VAL_DIR, cls_name)
        if os.path.isdir(cls_path):
            images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                example_path = os.path.join(cls_path, images[0])
                example_label = cls_name
                break

if example_path:
    print("Single Image Grad-CAM Analysis")
    print("=" * 50)
    pred_label, confidence, heatmap = visualize_gradcam_single(
        example_path, 
        true_label=example_label,
        alpha=0.5
    )
else:
    print("No validation images found. Skipping single image test.")

In [ ]:
# Visualize from CSV predictions file (if available)
import pandas as pd
import os

if os.path.exists(csv_path):
    df = pd.read_csv(csv_path)
    print(f"Loaded {len(df)} predictions from CSV")
    
    # Sample diverse images from CSV
    sample_df = df.groupby('true_label', group_keys=False).apply(
        lambda x: x.sample(min(2, len(x)), random_state=42)
    ).reset_index(drop=True)
    
    sample_df = sample_df.head(6)  # Limit to 6 total
    
    print(f"\nVisualizing {len(sample_df)} samples from validation predictions:")
    print("=" * 70)
    
    for idx, row in sample_df.iterrows():
        print(f"\n[{idx+1}/{len(sample_df)}] {os.path.basename(row['filepath'])}")
        visualize_gradcam_single(
            row['filepath'], 
            true_label=row['true_label'],
            alpha=0.5
        )
        print("-" * 70)
else:
    print(f"CSV file not found at: {csv_path}")
    print("Run the prediction cell first to generate the CSV.")

In [ ]:
# # Cell 13 — Fine-tune some top EfficientNet layers
# # The EfficientNet layers are directly in model.layers (not nested) when using input_tensor

# # Method 1: Find EfficientNet layers by name pattern
# efficientnet_layers = [l for l in model.layers if 'efficientnet' in l.name.lower() or 
#                        l.name.startswith('block') or 
#                        l.name.startswith('stem') or
#                        l.name == 'top_conv' or
#                        l.name == 'top_bn']

# # Method 2: More robust - find by layer type patterns common in EfficientNet
# if not efficientnet_layers:
#     efficientnet_layers = [l for l in model.layers 
#                           if any(x in l.name for x in ['block', 'stem', 'top_conv', 'top_bn', 'mbconv'])]

# print(f"Found {len(efficientnet_layers)} EfficientNet-related layers")

# if len(efficientnet_layers) == 0:
#     print("No EfficientNet layers found. Listing all model layers:")
#     for i, l in enumerate(model.layers):
#         print(f"  {i}: {l.name} ({l.__class__.__name__})")
#     print("\nSkipping fine-tune.")
# else:
#     # Print which layers we found
#     print("EfficientNet layers found (last 10):")
#     for l in efficientnet_layers[-10:]:
#         print(f"  - {l.name} (trainable: {l.trainable})")
    
#     # Unfreeze the last N EfficientNet layers (avoid BatchNorm)
#     UNFREEZE_COUNT = 40  # unfreeze last 40 EfficientNet layers
#     layers_to_unfreeze = efficientnet_layers[-UNFREEZE_COUNT:]
    
#     unfrozen_count = 0
#     for layer in layers_to_unfreeze:
#         # Skip BatchNormalization layers (keep them frozen for stability)
#         if not isinstance(layer, tf.keras.layers.BatchNormalization):
#             layer.trainable = True
#             unfrozen_count += 1
    
#     print(f"\n✓ Unfroze {unfrozen_count} layers for fine-tuning")
    
#     # Also ensure our custom layers are trainable
#     for layer in model.layers:
#         if layer.name in ['gradcam_target_conv', 'gradcam_target_bn', 'feature_conv']:
#             layer.trainable = True
#             print(f"  + {layer.name} set to trainable")
    
#     # Compile with a lower learning rate
#     model.compile(
#         optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
#         loss='sparse_categorical_crossentropy',
#         metrics=['accuracy']
#     )
    
#     print("\n📋 Model compiled for fine-tuning with lr=1e-5")
    
#     # Fine-tune training
#     ft_ckpt = os.path.join(MODEL_DIR, "best_model_finetuned.h5")
#     ft_history = model.fit(
#         train_ds,
#         validation_data=val_ds,
#         epochs=10,
#         class_weight=class_weight,
#         callbacks=[
#             callbacks.ModelCheckpoint(ft_ckpt, monitor='val_accuracy', save_best_only=True),
#             callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
#         ]
#     )
#     print("✓ Fine-tune complete. Best weights saved to:", ft_ckpt)

# Cell 13 — Fine-tune some top EfficientNet layers
# The EfficientNet layers are directly in model.layers (not nested) when using input_tensor

from tensorflow.keras import callbacks  # ensure callbacks imported

# Method 1: Find EfficientNet layers by name pattern
efficientnet_layers = [
    l for l in model.layers
    if 'efficientnet' in l.name.lower()
    or l.name.startswith('block')
    or l.name.startswith('stem')
    or l.name == 'top_conv'
    or l.name == 'top_bn'
]

# Method 2: More robust - find by layer type/name patterns common in EfficientNet
if not efficientnet_layers:
    efficientnet_layers = [
        l for l in model.layers
        if any(x in l.name for x in ['block', 'stem', 'top_conv', 'top_bn', 'mbconv'])
    ]

print(f"Found {len(efficientnet_layers)} EfficientNet-related layers")

if len(efficientnet_layers) == 0:
    print("No EfficientNet layers found. Listing all model layers:")
    for i, l in enumerate(model.layers):
        print(f"  {i}: {l.name} ({l.__class__.__name__})")
    print("\nSkipping fine-tune.")
else:
    # Print which layers we found
    print("EfficientNet layers found (last 10):")
    for l in efficientnet_layers[-10:]:
        print(f"  - {l.name} (trainable: {l.trainable})")
    
    # Unfreeze the last N EfficientNet layers (avoid BatchNorm)
    UNFREEZE_COUNT = 40  # unfreeze last 40 EfficientNet layers
    layers_to_unfreeze = efficientnet_layers[-UNFREEZE_COUNT:]
    
    unfrozen_count = 0
    for layer in layers_to_unfreeze:
        # Skip BatchNormalization layers (keep them frozen for stability)
        if not isinstance(layer, tf.keras.layers.BatchNormalization):
            layer.trainable = True
            unfrozen_count += 1
    
    print(f"\n✓ Unfroze {unfrozen_count} layers for fine-tuning")
    
    # Also ensure our custom layers are trainable
    for layer in model.layers:
        if layer.name in ['gradcam_target_conv', 'gradcam_target_bn', 'feature_conv']:
            layer.trainable = True
            print(f"  + {layer.name} set to trainable")
    
    # Compile with a lower learning rate for fine-tuning
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    print("\n📋 Model compiled for fine-tuning with lr=1e-5")
    
    # Fine-tune training — use new Keras format for checkpoint
    ft_ckpt = os.path.join(MODEL_DIR, "best_model_finetuned.keras")

    ft_callbacks = [
        callbacks.ModelCheckpoint(
            filepath=ft_ckpt,
            monitor='val_accuracy',
            save_best_only=True,
            save_weights_only=False,
            mode='max',
            verbose=1
        ),
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            mode='min',
            verbose=1
        ),
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=1e-7,
            mode='min',
            verbose=1
        )
    ]

    ft_history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=10,
        class_weight=class_weight,
        callbacks=ft_callbacks
    )
    print("✓ Fine-tune complete. Best weights saved to:", ft_ckpt)



In [ ]:
# Quick test prediction
# Find an example image from training set

import os

test_img_path = None
if os.path.exists(TRAIN_DIR):
    for cls_name in class_names:
        cls_path = os.path.join(TRAIN_DIR, cls_name)
        if os.path.isdir(cls_path):
            images = [f for f in os.listdir(cls_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            if images:
                test_img_path = os.path.join(cls_path, images[0])
                break

if test_img_path:
    img_batch = load_and_preprocess_for_model(test_img_path)
    preds = model.predict(img_batch)
    print(f"Test image: {os.path.basename(test_img_path)}")
    print(f"Predictions: {preds}")
    print(f"Predicted class: {class_names[np.argmax(preds)]} (index: {np.argmax(preds)})")
else:
    print("No test image found.")

In [ ]:
# # Cell 14 — Save final model and plots
# final_model_path = os.path.join(MODEL_DIR, "final_model.h5")
# model.save(final_model_path)
# print("Saved final model to:", final_model_path)

# # Plot history if available (initial training)
# def plot_history(history, fname_prefix="history"):
#     import matplotlib.pyplot as plt
#     hist = history.history
#     if 'accuracy' in hist:
#         plt.figure(figsize=(8,4))
#         plt.plot(hist['accuracy'], label='train_acc')
#         plt.plot(hist['val_accuracy'], label='val_acc')
#         plt.title('Accuracy')
#         plt.legend()
#         plt.savefig(os.path.join(MODEL_DIR, f"{fname_prefix}_acc.png"))
#         plt.close()
#     if 'loss' in hist:
#         plt.figure(figsize=(8,4))
#         plt.plot(hist['loss'], label='train_loss')
#         plt.plot(hist['val_loss'], label='val_loss')
#         plt.title('Loss')
#         plt.legend()
#         plt.savefig(os.path.join(MODEL_DIR, f"{fname_prefix}_loss.png"))
#         plt.close()

# # call for initial training history
# try:
#     plot_history(history, "initial")
# except NameError:
#     pass

# try:
#     plot_history(ft_history, "finetune")
# except NameError:
#     pass

# print("Saved training plots (if histories exist) into:", MODEL_DIR)



# Cell 14 — Save final model and plots
final_model_path = os.path.join(MODEL_DIR, "final_model.keras")  # ✔ modern format
model.save(final_model_path)
print("Saved final model to:", final_model_path)

# Plot history if available
def plot_history(history, fname_prefix="history"):
    import matplotlib.pyplot as plt
    hist = history.history

    if 'accuracy' in hist:
        plt.figure(figsize=(8,4))
        plt.plot(hist['accuracy'], label='train_acc')
        plt.plot(hist.get('val_accuracy', []), label='val_acc')
        plt.title('Accuracy')
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(MODEL_DIR, f"{fname_prefix}_acc.png"))
        plt.close()

    if 'loss' in hist:
        plt.figure(figsize=(8,4))
        plt.plot(hist['loss'], label='train_loss')
        plt.plot(hist.get('val_loss', []), label='val_loss')
        plt.title('Loss')
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.tight_layout()
        plt.savefig(os.path.join(MODEL_DIR, f"{fname_prefix}_loss.png"))
        plt.close()

# Safely plot training histories (if they exist)
try:
    plot_history(history, "initial")
except NameError:
    pass

try:
    plot_history(ft_history, "finetune")
except NameError:
    pass

print("Saved training plots (if histories exist) into:", MODEL_DIR)


In [ ]:
# Cell 15 — Notes for RAG Integration
# Use the predictions CSV produced at:
#   {ARTIFACTS_DIR}/val_predictions.csv
# For each row you have: filepath, true_label, pred_label, confidence
# 
# RAG ingestion steps (high-level):
# 1. Build an index (FAISS) of literature + prior reports (text chunks).
# 2. For each prediction row, retrieve top-k supporting text snippets from the index.
# 3. Create a prompt containing: anonymized patient id, pred_label & confidence, top-k retrievals, any Grad-CAM notes.
# 4. Send the prompt to the LLM to generate a structured clinical report.
# 5. Use n8n to automate the pipeline: webhook -> inference -> retrieval -> LLM -> save/send report.

print("✓ Notebook complete!")
print(f"Models saved to: {MODEL_DIR}")
print(f"Reports/artifacts saved to: {REPORT_DIR} and {ARTIFACTS_DIR}")
print(f"Predictions CSV: {os.path.join(ARTIFACTS_DIR, 'val_predictions.csv')}")